# Importation des Librairies 

In [1]:
import pandas as pd
import os
import pyAgrum as gum
from pyagrum_extra import gum
import dash
from dash import dcc
from dash import html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px

## Import Data

In [2]:
ot_odr_filename = os.path.join("./data", "OT_ODR.csv.bz2")
df = pd.read_csv(ot_odr_filename,
                        compression="bz2",
                        sep=";")

In [3]:
df

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,EQU_ID,DATE_OT,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE
0,OT000000000,OM000000000,REMPLACEMENT D'UNE GLACE LAT VOYAGEUR,CARROSSERIE,4.00,EQUIPEMENT DE CARROSSERIE,VITRAGE,VITRAGE LAT,E00005934,2011-03-29 19:26:06,149698.557783,GLACE/BAIE,INTERIEUR/GAUCHE/ARRIERE,DEBOITE,L0482
1,OT000000001,OM000000001,REMPLACEMENT D'UN COMMODO DE SIGNALISATION,ELECTRICITE,0.50,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00004713,2011-05-03 20:01:31,225035.016000,KLAXON/GONG,AVANT,ABSENT,L0147
2,OT000000002,OM000000002,REMPLACEMENT D'UN CARDAN DE LIAISON SUR CREMAI...,MECANIQUE,1.50,EQUIPEMENT CHASSIS,EQUIPEMENT DE DIRECTION,COMMANDE DE DIRECTION,E00006037,2011-05-05 14:40:22,71148.834963,VOITURE,A L'ACCELERATION,VIBRE,L0368
3,OT000000003,OM000000003,REMPLACEMENT D'UN PARE-CHOCS AVG,CARROSSERIE,0.50,EQUIPEMENT DE CARROSSERIE,ELEMENT CARROSSERIE EXT,PROTECTION AV,E00005670,2011-05-07 07:43:27,116441.657358,PARE-CHOCS,AVANT/GAUCHE,CASSE,L0066
4,OT000000004,OM000000004,REMPLACEMENT D'UN POTENTIOMETRE DE PORTE NUMERO 1,ELECTRICITE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,COMMANDE PORTE,E00004009,2011-05-18 10:56:50,0.000000,SECURITE PORTE,ARRIERE,BLOQUE,L0247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,OT000405952,OM000506539,REMPLACEMENT D'UN FEU DE RECUL,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00040793,2019-09-30 20:38:09,296005.373397,ECLAIRAGE FEUX EXTERIEURS,ARRIERE/EXTERIEUR/DROIT,CASSE,L0283
506554,OT000405953,OM000506546,REMPLACEMENT D'UN ECLAIRAGE EXT,MECANIQUE,0.75,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00274690,2019-09-30 21:21:20,153050.080050,ECLAIRAGE FEUX EXTERIEURS,ARRIERE/LATERAL,CASSE,L0116
506555,OT000405954,OM000506536,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.03,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00256452,2019-09-30 21:39:29,175063.182439,AVTT,AVANT/PORTE,INTERMITTENT,L0134
506556,OT000405955,OM000506545,REMPLACEMENT D'UNE LAMPE DE FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00006122,2019-09-30 21:55:28,437053.614263,ECLAIRAGE FEUX EXTERIEURS,LATERAL/HAUT/GAUCHE,NE FONCTIONNE PAS,L0270


## One Hot Encoding - SIG_CONTEXTE

In [4]:
# Créer le DataFrame à partir des colonnes
df_contexte = pd.DataFrame(df['SIG_CONTEXTE'])

# Diviser les valeurs de la colonne "SIG_CONTEXTE" en colonnes distinctes si elles n'existent pas déjà
if 'SIG_CONTEXTE' in df_contexte.columns:
    split_values = df_contexte["SIG_CONTEXTE"].str.split("/", expand=True)

# Appliquer l'encodage one-hot aux colonnes distinctes
one_hot_encoded = pd.get_dummies(split_values)

# Regrouper les colonnes encodées
grouped_encoded = one_hot_encoded.groupby(lambda x: x.split("_")[1], axis=1).sum()

# Ajouter un préfixe aux nouvelles colonnes
grouped_encoded = grouped_encoded.add_prefix("SIG_CONTEXTE=")

# Concaténer les colonnes encodées avec le DataFrame d'origine
df_contexte = pd.concat([df_contexte, grouped_encoded], axis=1)

# Concaténation df_merger, df_contexte
df = pd.concat([df, df_contexte], axis=1)

df

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,EQU_ID,DATE_OT,...,SIG_CONTEXTE=HAUT,SIG_CONTEXTE=INTERIEUR,SIG_CONTEXTE=LATERAL,SIG_CONTEXTE=PLAFOND,SIG_CONTEXTE=PLATE FORME,SIG_CONTEXTE=PORTE,SIG_CONTEXTE=POSTE CONDUITE,SIG_CONTEXTE=REMORQUE,SIG_CONTEXTE=ROTONDE,SIG_CONTEXTE=TABLEAU DE BORD
0,OT000000000,OM000000000,REMPLACEMENT D'UNE GLACE LAT VOYAGEUR,CARROSSERIE,4.00,EQUIPEMENT DE CARROSSERIE,VITRAGE,VITRAGE LAT,E00005934,2011-03-29 19:26:06,...,0,1,0,0,0,0,0,0,0,0
1,OT000000001,OM000000001,REMPLACEMENT D'UN COMMODO DE SIGNALISATION,ELECTRICITE,0.50,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00004713,2011-05-03 20:01:31,...,0,0,0,0,0,0,0,0,0,0
2,OT000000002,OM000000002,REMPLACEMENT D'UN CARDAN DE LIAISON SUR CREMAI...,MECANIQUE,1.50,EQUIPEMENT CHASSIS,EQUIPEMENT DE DIRECTION,COMMANDE DE DIRECTION,E00006037,2011-05-05 14:40:22,...,0,0,0,0,0,0,0,0,0,0
3,OT000000003,OM000000003,REMPLACEMENT D'UN PARE-CHOCS AVG,CARROSSERIE,0.50,EQUIPEMENT DE CARROSSERIE,ELEMENT CARROSSERIE EXT,PROTECTION AV,E00005670,2011-05-07 07:43:27,...,0,0,0,0,0,0,0,0,0,0
4,OT000000004,OM000000004,REMPLACEMENT D'UN POTENTIOMETRE DE PORTE NUMERO 1,ELECTRICITE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,COMMANDE PORTE,E00004009,2011-05-18 10:56:50,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,OT000405952,OM000506539,REMPLACEMENT D'UN FEU DE RECUL,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00040793,2019-09-30 20:38:09,...,0,0,0,0,0,0,0,0,0,0
506554,OT000405953,OM000506546,REMPLACEMENT D'UN ECLAIRAGE EXT,MECANIQUE,0.75,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00274690,2019-09-30 21:21:20,...,0,0,1,0,0,0,0,0,0,0
506555,OT000405954,OM000506536,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.03,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00256452,2019-09-30 21:39:29,...,0,0,0,0,0,1,0,0,0,0
506556,OT000405955,OM000506545,REMPLACEMENT D'UNE LAMPE DE FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00006122,2019-09-30 21:55:28,...,1,0,1,0,0,0,0,0,0,0


## MODELE 3 : 

In [5]:
colonnes_sig_contexte = df.filter(regex=r'^SIG_CONTEXTE=')

# Obtenez les noms des colonnes
noms_colonnes_sig_contexte = colonnes_sig_contexte.columns.tolist()

In [6]:
var_to_model = ["SYSTEM_N1", "SIG_OBS", "SYSTEM_N2","SIG_ORGANE","SYSTEM_N3"] + noms_colonnes_sig_contexte

var_bn = {}
for var in var_to_model:
    df[var] = df[var].astype('category')  # Conversion en type 'category'
    nb_values = len(df[var].cat.categories)
    var_bn[var] = gum.LabelizedVariable(var, var, nb_values)

### Ajout les labels

In [7]:
for var in var_bn:
    for i, modalite in enumerate(df[var].cat.categories):
        var_bn[var].changeLabel(i, str(modalite))

### Création du RB

In [8]:
var_to_model2 = ["SIG_OBS", "SYSTEM_N2","SIG_ORGANE"] + noms_colonnes_sig_contexte

bn = gum.BayesNet("Model 3")

for var in var_bn.values():
    bn.add(var)

for val in var_to_model2:
    bn.addArc("SYSTEM_N1", val)
bn.addArc("SYSTEM_N2", "SYSTEM_N3")

bn

(pyAgrum.BayesNet<double>@0x105463800) BN{nodes: 39, arcs: 38, domainSize: 10^18.8996, dim: 9224, mem: 75Ko 320o}

### Fit le RB pour calculer les probabilités

In [9]:
bn.fit_bis(df, verbose_mode=True)

- Learn CPT SIG_CONTEXTE=DESSOUS
- Learn CPT SIG_CONTEXTE=REMORQUE
- Learn CPT SIG_CONTEXTE=BAS
- Learn CPT SIG_ORGANE
- Learn CPT SIG_CONTEXTE=EN VIRAGE
- Learn CPT SIG_CONTEXTE=CENTRE
- Learn CPT SIG_CONTEXTE=PLAFOND
- Learn CPT SIG_CONTEXTE=A L'ARRET
- Learn CPT SIG_CONTEXTE=ARRIERE
- Learn CPT SIG_CONTEXTE=AU RALENTI
- Learn CPT SIG_CONTEXTE=EXTERIEUR
- Learn CPT SIG_CONTEXTE=A CHAUD
- Learn CPT SYSTEM_N1
- Learn CPT SIG_CONTEXTE=HAUT
- Learn CPT SIG_CONTEXTE=DROIT
- Learn CPT SIG_CONTEXTE=TABLEAU DE BORD
- Learn CPT SIG_CONTEXTE=A LA FERMETURE
- Learn CPT SIG_CONTEXTE=A L'OUVERTURE
- Learn CPT SYSTEM_N2
- Learn CPT SIG_CONTEXTE=AU DEMARRAGE
- Learn CPT SIG_CONTEXTE=AU FREINAGE
- Learn CPT SIG_OBS
- Learn CPT SIG_CONTEXTE=CENTRALE
- Learn CPT SIG_CONTEXTE=PORTE
- Learn CPT SIG_CONTEXTE=A L'ACCELERATION
- Learn CPT SIG_CONTEXTE=A VIDE
- Learn CPT SIG_CONTEXTE=EN DESCENTE
- Learn CPT SIG_CONTEXTE=LATERAL
- Learn CPT SIG_CONTEXTE=ROTONDE
- Learn CPT SIG_CONTEXTE=A FROID
- Learn CPT SI

In [10]:
bn.cpt("SYSTEM_N1")

(pyAgrum.Potential<double>@0x11db3b380) 
  SYSTEM_N1                                                                                        |
DIVERS   |EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|
---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|
 0.0001  | 0.0316  | 0.0270  | 0.2472  | 0.0232  | 0.0450  | 0.0027  | 0.5633  | 0.0405  | 0.0195  |

# Performance Modèle

In [11]:
var_features = ["SIG_OBS", "SIG_ORGANE"] + noms_colonnes_sig_contexte # Variables explicatives
var_targets = ["SYSTEM_N1","SYSTEM_N2","SYSTEM_N3"] # Variables à expliquer

In [12]:
def evaluation(list_var_obsr: list,var_target: str):
    # Pour 30% des valeurs mettres : 151966 
    df_train = df.iloc[:-10000]
    df_test = df.iloc[-10000:]
    bn.fit_bis(df_train, verbose_mode=True)
    pred = bn.predict(df_test[list_var_obsr], var_target=var_target, show_progress=True)
    evaluation = (df_test[var_target] == pred).mean()
    print(f"Pour la target '{var_target}' avec les variables observés : {list_var_obsr} \nCe modèle on obtiens une performance de {evaluation*100} %")
    return evaluation

evaluation_1 = evaluation(var_features,"SYSTEM_N1")
evaluation_2 = evaluation(["SYSTEM_N1"],"SYSTEM_N2")
evaluation_3 = evaluation(["SYSTEM_N2"],"SYSTEM_N3")

- Learn CPT SIG_CONTEXTE=DESSOUS
- Learn CPT SIG_CONTEXTE=REMORQUE
- Learn CPT SIG_CONTEXTE=BAS
- Learn CPT SIG_ORGANE
- Learn CPT SIG_CONTEXTE=EN VIRAGE
- Learn CPT SIG_CONTEXTE=CENTRE
- Learn CPT SIG_CONTEXTE=PLAFOND
- Learn CPT SIG_CONTEXTE=A L'ARRET
- Learn CPT SIG_CONTEXTE=ARRIERE
- Learn CPT SIG_CONTEXTE=AU RALENTI
- Learn CPT SIG_CONTEXTE=EXTERIEUR
- Learn CPT SIG_CONTEXTE=A CHAUD
- Learn CPT SYSTEM_N1
- Learn CPT SIG_CONTEXTE=HAUT
- Learn CPT SIG_CONTEXTE=DROIT
- Learn CPT SIG_CONTEXTE=TABLEAU DE BORD
- Learn CPT SIG_CONTEXTE=A LA FERMETURE
- Learn CPT SIG_CONTEXTE=A L'OUVERTURE
- Learn CPT SYSTEM_N2
- Learn CPT SIG_CONTEXTE=AU DEMARRAGE
- Learn CPT SIG_CONTEXTE=AU FREINAGE
- Learn CPT SIG_OBS
- Learn CPT SIG_CONTEXTE=CENTRALE
- Learn CPT SIG_CONTEXTE=PORTE
- Learn CPT SIG_CONTEXTE=A L'ACCELERATION
- Learn CPT SIG_CONTEXTE=A VIDE
- Learn CPT SIG_CONTEXTE=EN DESCENTE
- Learn CPT SIG_CONTEXTE=LATERAL
- Learn CPT SIG_CONTEXTE=ROTONDE
- Learn CPT SIG_CONTEXTE=A FROID
- Learn CPT SI

In [13]:
performance_data = [
    {
        'Target': var_targets[0],
        'Pourcentage de performance': evaluation_1 * 100
    },
    {
        'Target': var_targets[0],
        'Pourcentage de performance': evaluation_1 * 100
    },
    {
        'Target': var_targets[2],
        'Pourcentage de performance': evaluation_3 * 100
    }
]
performance_df = pd.DataFrame(performance_data)


In [ ]:
# Extraction des valeurs de la colonne "SIG_CONTEXTE"
sig_contexte_values = [col.replace('SIG_CONTEXTE=', '') for col in noms_colonnes_sig_contexte]

# Filtrage des variables pour inclure uniquement "SIG_OBS", "SIG_ORGANE" et "SIG_CONTEXTE"
var_features_filtered = ["SIG_OBS", "SIG_ORGANE", "SIG_CONTEXTE"]

## Application Dash

In [14]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Modèle 3"),
    html.Div([
        html.Div([
            html.Label(var),
            dcc.Dropdown(
                id=f'{var}-dropdown',
                options=[{'label': i, 'value': i} for i in (sig_contexte_values if var == "SIG_CONTEXTE" else df[var].cat.categories)],
                value=(None if var == "SIG_CONTEXTE" else df[var].cat.categories[0]),
                multi=(var == "SIG_CONTEXTE")
            )
        ]) for var in var_features_filtered
    ], style={'width': '30%', 'display': 'inline-block'}),
    html.Div([
        dash_table.DataTable(
            id='performance-table',
            columns=[{"name": i, "id": i} for i in performance_df.columns],
            data=performance_df.to_dict('records'),
        ),
    ], style={'margin': '20px'}),
    html.Div([
        dcc.Graph(id=f'{var}-graph', config={'displayModeBar': False})
        for var in var_targets
    ], style={'width': '65%', 'float': 'right', 'display': 'inline-block'})
])

### Update graph with selected options

In [16]:
@app.callback(
    [Output(f'{var}-graph', 'figure') for var in var_targets],
    [Input(f'{var}-dropdown', 'value') for var in var_features_filtered]
)
def update_graph(sig_obs_value, sig_organe_value, sig_contexte_values):
    bn_ie = gum.LazyPropagation(bn)

    ev = {"SIG_OBS": sig_obs_value, "SIG_ORGANE": sig_organe_value}
    for value in sig_contexte_values:
        ev[f"SIG_CONTEXTE={value}"] = 1

    bn_ie.setEvidence(ev)
    bn_ie.makeInference()

    prob_target = []
    for i, var in enumerate(var_targets):
        prob_target_var = bn_ie.posterior(var).topandas().droplevel(0)
        prob_target_var_sorted = prob_target_var.sort_values(ascending=False)
        top_5_probs = prob_target_var_sorted.head(5)
        prob_fig = px.bar(top_5_probs)
        prob_fig.update_layout(title=f"Top 5 Probabilités pour {var}")
        prob_fig.update_xaxes(title="Catégorie")
        prob_fig.update_yaxes(title="Probabilités")
        prob_target.append(prob_fig)
    return tuple(prob_target)

In [17]:
app.run_server(debug=True, port=8049, use_reloader=False)

Dash is running on http://127.0.0.1:8049/

 * Serving Flask app '__main__'
 * Debug mode: on
